- 해당 모델은 단어 토큰을 입력했을 때 출력되는 NER 태그값을 예측하는 문제
- 예) '삼성전자'를 입력했을 때 단체를 뜻하는 B_OG(organization)태그가 출력되도록 모델을 학습화면 됨

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
#학습 파일 불러오기
def read_file(file_name):
    sents= []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

#학습 데이터 구조에 맞게 파일을 읽어와 문장 라인별로
#토큰 번호, 단어 토큰, 품사 태그, bio 태그 정보를 불러옴

In [4]:
#학습용 말뭉치 데이터 불러오기 - 1
corpus = read_file('./KoreanNERCorpus-master/original/train.txt')

In [5]:
#말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성 - 2
sentences, tags = [],[]
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1],w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    sentences.append(sentence)
    tags.append(bio_tag)
print('샘플크기 :\n', len(sentences))
print('0번째 샘플 문장 시퀀스: \n', sentences[0]) #단어 시퀀스
print('0번째 bio 태그: \n', tags[0]) #bio 태그 정보들이 tags에 저장됨
print('샘플 문장 시퀀스 최대 길이: ', max(len(l) for l in sentences )) 
print('샘플 문장 시퀀스 평균 길이:', (sum(map(len, sentences)) / len(sentences)))

#1에서 불러온 말뭉치 데이터에서 단어와 bio 태그만 이용해 학습용 데이터셋을 생성함
#0번째 문장을 예시로 출력

#0번째 원본 문장에서 분리된 단어 토큰들이 sentences리스트에 저장됨

#sentences리스트와 tags 리스트의 크기는 동일함

샘플크기 :
 3555
0번째 샘플 문장 시퀀스: 
 ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 bio 태그: 
 ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이:  168
샘플 문장 시퀀스 평균 길이: 34.03909985935302


In [6]:
#토크나이저 정의 - 3
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')#첫 번째 인덱스에는 oov 사용
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower = False) # 태그 정보는 lower = False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

#2에서 만들어진 단어 시퀀스와 태그 시퀀스를 사전으로 만들기 위해 토크나이저를 정의한 후 fit_on_texts() 함수 호출
#oov는 out of vacabulary의 약자로 단어 사전에 포함되지 않는 단어를 의미함
#단어 사전의 첫 번째 인덱스 토큰값으로 oov를 설정함 

In [7]:
#단어 사전 및 태그 사전 크기 정의
vocab_size = len(sent_tokenizer.word_index) + 1
tag_size =len(tag_tokenizer.word_index) + 1
print('bio 태그 사전 크기:', tag_size)
print('단어 사전 크기:',vocab_size)

bio 태그 사전 크기: 8
단어 사전 크기: 13834


In [8]:
#학습용 단어 시퀀스 생성 - 4
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[0])
print(y_train[0])

#3에서 만들어진 사전 데이터를 시퀀스 번호 형태로 인코딩

[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3]
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [9]:
# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = 'PAD'

In [10]:
# 시퀀스 패딩 처리 -5 
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

#개체명 인식 모델의 입출력 벡터 크기를 동일하게 맞추기 위해 시퀀스 패딩 작업하기

In [11]:
#학습 데이터와 테스트 데이터를 8:2 비율로 분리 - 6
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [12]:
# 출력 데이터를 원핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes= tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes= tag_size)

print('학습 샘플 시퀀스 형상:', x_train.shape)
print('학습 샘플 레이블 형상:', y_train.shape)

print('테스트 샘플 시퀀스 형상:', x_test.shape)
print('테스트 샘플 레이블 형상:', y_test.shape)

#학습과 테스트용 출력 데이터(y_train, t_test)를 태그 사전 크기에 맞게 원핫 인코딩 하기

학습 샘플 시퀀스 형상: (2844, 40)
학습 샘플 레이블 형상: (2844, 40, 8)
테스트 샘플 시퀀스 형상: (711, 40)
테스트 샘플 레이블 형상: (711, 40, 8)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

In [19]:
#모델 정의(Bi-LSTM) - 7
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout = 0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)

#개체 인식 모델을 순차 모델 방식으로 구현함
#tag_size만큼의 출력 뉴런에서 제일 확률 높은 출력값 1개를 선택하는 문제이기 때문에 softmax,categorical_crossentropy사용
#

Epoch 1/10
23/23 [==============================] - 15s 413ms/step - loss: 0.6984 - accuracy: 0.8402
Epoch 2/10
23/23 [==============================] - 11s 468ms/step - loss: 0.3268 - accuracy: 0.8973
Epoch 3/10
23/23 [==============================] - 11s 473ms/step - loss: 0.2286 - accuracy: 0.9220
Epoch 4/10
23/23 [==============================] - 11s 499ms/step - loss: 0.1579 - accuracy: 0.9470
Epoch 5/10
23/23 [==============================] - 12s 522ms/step - loss: 0.1127 - accuracy: 0.9635
Epoch 6/10
23/23 [==============================] - 12s 535ms/step - loss: 0.0824 - accuracy: 0.9743
Epoch 7/10
23/23 [==============================] - 12s 532ms/step - loss: 0.0612 - accuracy: 0.9808
Epoch 8/10
23/23 [==============================] - 12s 537ms/step - loss: 0.0491 - accuracy: 0.9845
Epoch 9/10
23/23 [==============================] - 13s 552ms/step - loss: 0.0395 - accuracy: 0.9874
Epoch 10/10
23/23 [==============================] - 13s 560ms/step - loss: 0.0339 - accura

In [20]:
print('평과 결과:', model.evaluate(x_test, y_test)[1])

23/23 [==============================] - 3s 112ms/step - loss: 0.2815 - accuracy: 0.9365
평과 결과: 0.9365341067314148


- 개체명 인식에 사용되는 성능 평가는 F1 스코어를 계산하는 방법을 사용해야함
- F1 스코어를 계산하기 위해서는 정밀도와 재현율을 사용해야함
    - 정확도(accuracy): 실제 정답과 얼마나 유사한지 나타냄
    - 정밀도(precision): 정밀도가 높다고 해서 정확하다는 의미는 아님 정밀도가 높으면 결과값이 일정하게 분포되어 있는 것
    - 재현율(recall): 실제 정답인 것들 중 예측 모델이 정답이라 예측한 것의 비율
- F1 스코어란 정밀도와 재현율의 조화 평균을 의미함

In [24]:
#시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result=[]
    for sequence in sequences:
        temp =[]
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace('PAD','O'))
        result.append(temp)
    return result

In [25]:
#테스트 데이터셋의 NER 예측 - 8
y_predicted = model.predict(x_test) # 711,40 -> model -> 711,40,8
pred_tags = sequences_to_tag(y_predicted) #실제 ner
tets_tags = sequences_to_tag(y_test) #예측 ner

23/23 [==============================] - 2s 104ms/step


In [32]:
#f1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report  
print(classification_report(tets_tags, pred_tags))
print('f1-score: {:.1%}'.format(f1_score(tets_tags, pred_tags)))

#seqeval.metrics 모듈의 classification_report함수를 통해 ner 태그별로 계산된 정밀도와 재현율,f1 스코어를 출력함

c:\Users\ETRI\anaconda3\envs\virtual\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\ETRI\anaconda3\envs\virtual\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\ETRI\anaconda3\envs\virtual\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\ETRI\anaconda3\envs\virtual\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\ETRI\anaconda3\envs\virtual\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

           _       0.57      0.57      0.57       657
         _DT       0.91      0.89      0.90       335
         _LC       0.76      0.53      0.63       312
         _OG       0.67      0.60      0.63       481
         _PS       0.75      0.48      0.59       374
         _TI       0.92      0.73      0.81        66

   micro avg       0.70      0.61      0.65      2225
   macro avg       0.76      0.63      0.69      2225
weighted avg       0.71      0.61      0.65      2225

f1-score: 65.3%


In [37]:
#새로운 유형의 문장 ner 예측 - 9
word_to_index = sent_tokenizer.word_index
new_sentence='삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.'.split()
new_x = []
for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w,1))
    except KeyError:
        #모르는 단어읭 경우 oov
        new_x.append(word_to_index['OOV'])
print('새로운 유형 시퀀스:', new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding='post', value=0,maxlen=max_len)

새로운 유형 시퀀스: [531, 307, 1476, 286, 1507, 6766, 1]


In [39]:
#ner 예측
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=-1) # 예측된 ner 인덱스값 추출
 
print('{:10} {:5}'.format('단어', '예측된 ner'))
print('-'*50)
for w, pred in zip(new_sentence, p[0]):
    print('{:10} {:5}'.format(w, index_to_ner[pred]))
    

#삼성전자, 애플은 B_OG(조직)
#오늘 B_DT(날짜)로 정확하게 판단
#애플 뒤 도전장, 내밀다에는 O로 되어야하는데 I로 잘못 판단함 

1/1 [==============================] - 0s 32ms/step
단어         예측된 ner
--------------------------------------------------
삼성전자       B_OG 
출시         O    
스마트폰       O    
오늘         B_DT 
애플         B_OG 
도전장        I    
내밀다.       I    
